<a href="https://colab.research.google.com/github/mawadaenn/US_accidentes/blob/chaimaa/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import pandas as pd

file = "/content/drive/MyDrive/Colab Notebooks/US_Accidents/US_Accidents.csv"
df = pd.read_csv(file)




In [8]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)       

In [9]:
# valeurs manquantes par colonne
valeurs_nulles = df.isnull().sum()
taux_valeurs_nulles = round((df.isnull().sum() / len(df)) * 100, 2)

df_nulles = pd.DataFrame({
    'Colonne': valeurs_nulles.index,
    'Nb_Manquants': valeurs_nulles.values,
    'Pourcentage': taux_valeurs_nulles.values
}).sort_values('Nb_Manquants', ascending=False)

print(df_nulles[df_nulles['Nb_Manquants'] > 0])

                  Colonne  Nb_Manquants  Pourcentage
8                 End_Lng       3402762        44.03
7                 End_Lat       3402762        44.03
27      Precipitation(in)       2203586        28.51
21          Wind_Chill(F)       1999019        25.87
26        Wind_Speed(mph)        571233         7.39
24         Visibility(mi)        177098         2.29
25         Wind_Direction        175206         2.27
22            Humidity(%)        174144         2.25
28      Weather_Condition        173459         2.24
20         Temperature(F)        163853         2.12
23           Pressure(in)        140679         1.82
19      Weather_Timestamp        120228         1.56
42         Sunrise_Sunset         23246         0.30
43         Civil_Twilight         23246         0.30
45  Astronomical_Twilight         23246         0.30
44      Nautical_Twilight         23246         0.30
18           Airport_Code         22635         0.29
11                 Street         10869       